**NOT:Dosya boyutu çok yüksek olduğu için çıktıları temizleyip yükledim, Run All yaparsanız hepsi geri gelecektir.**

### **Üsküdar Üniversitesi**

**Fen Bilimleri Enstitüsü**

**Yapay Zeka Mühendisliği Yüksek Lisans Programı**

**Sayısal Görüntü İşleme Ara Sınav Ödevi**

**Muhammed TURAN 244312007**

**Öğretim Üyesi: Dr. Gökalp TULUM**


## **Veri Yükleme**


**Kütüphanelerin İçe Aktarılması**




In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from PIL import Image

!pip install gdown
import gdown



file_id = '1nXVAxiA1iIhyUNLl33Epn-jgAQDRrAGu'


url = f'https://drive.google.com/uc?id=1nXVAxiA1iIhyUNLl33Epn-jgAQDRrAGu'
output_zip = 'dataset.zip'


print("Veri seti indiriliyor...")
gdown.download(url, output_zip, quiet=False)


print("Dosyalar çıkartılıyor...")

!unzip -q {output_zip} -d "Dataset_Klasoru"


!rm {output_zip}

print("İşlem tamamlandı!")



print("Kütüphaneler başarıyla yüklendi.")

**Veri Setinin Yüklenmesi**

In [ ]:

def load_image_paths(root_folder):

    image_paths = []
    labels = []


    for dirname, _, filenames in os.walk(root_folder):
        for filename in filenames:

            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                full_path = os.path.join(dirname, filename)
                image_paths.append(full_path)


                label = os.path.basename(dirname)
                labels.append(label)


    df = pd.DataFrame({
        'image_path': image_paths,
        'label': labels
    })

    return df

dataset_path = "Dataset_Klasoru/Skin cancer ISIC The International Skin Imaging Collaboration/Train"
train_df = load_image_paths(dataset_path)

print("Veri Seti İlk 5 Satır:")
print(train_df.head())


print(f"\nToplam Görüntü Sayısı: {len(train_df)}")


**Veri Özelliklerinin İncelenmesi**


In [ ]:
# Analiz sonuçlarını tutacak listeler
widths = []
heights = []
channels_rgb = []
channels_gray = []
file_sizes_kb = []

# İşlem süresini kısaltmak için ilk 500 görüntü üzerinde analiz yapalım
sample_df = train_df.head(500)

print("Görüntü özellikleri analiz ediliyor, lütfen bekleyin...")

for index, row in sample_df.iterrows():
    path = row['image_path']

    # 1. Dosya Boyutu (KB cinsinden)
    size_bytes = os.path.getsize(path)
    file_sizes_kb.append(size_bytes / 1024)

    # 2. Görüntüyü Okuma (OpenCV varsayılan olarak BGR okur, yani Renklidir)
    img = cv2.imread(path)

    if img is not None:
        # Şekil (Height, Width, Channels)
        h, w, c = img.shape
        heights.append(h)
        widths.append(w)
        channels_rgb.append(c)

        # 3. Grayscale Dönüşümü ve Kanal Kontrolü
        # Ödevde her iki format istendiği için griye çevirip kanalına bakıyoruz
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Grayscale görüntülerde genelde (H, W) döner, kanal boyutu düşer.
        # Boyut kontrolü:
        if len(gray_img.shape) == 2:
            channels_gray.append(1) # Tek kanal
        else:
            channels_gray.append(gray_img.shape[2])

# Sonuçları yeni bir DataFrame'e aktaralım (Analiz kolaylığı için)
analysis_df = pd.DataFrame({
    'Width': widths,
    'Height': heights,
    'RGB_Channels': channels_rgb,
    'Gray_Channels': channels_gray,
    'File_Size_KB': file_sizes_kb
})

# --- İSTATİSTİKLER VE GÖRSELLEŞTİRME ---

# 1. Çözünürlük Analizi
print("\n--- Çözünürlük İstatistikleri ---")
print(analysis_df[['Width', 'Height']].describe())

plt.figure(figsize=(10, 5))
sns.scatterplot(data=analysis_df, x='Width', y='Height', alpha=0.5)
plt.title("Görüntü Çözünürlük Dağılımı (Genişlik vs Yükseklik)")
plt.xlabel("Genişlik (px)")
plt.ylabel("Yükseklik (px)")
plt.show()

# 2. Kanal Sayısı Doğrulaması
print("\n--- Kanal Sayıları ---")
print(f"Orijinal (RGB) Kanal Ortalaması: {analysis_df['RGB_Channels'].mean()}")
print(f"Dönüştürülmüş (Gray) Kanal Ortalaması: {analysis_df['Gray_Channels'].mean()}")

# 3. Dosya Boyutu Dağılımı
plt.figure(figsize=(10, 5))
sns.histplot(analysis_df['File_Size_KB'], kde=True, bins=30)
plt.title("Dosya Boyutu Dağılımı (KB)")
plt.xlabel("Boyut (KB)")
plt.show()

**Çözünürlük:** Standart değil, çok değişken. Görüntüler 600x450'den başlayıp 3872x2592'ye kadar çıkıyor; ancak çoğunluk 1024x768 pikselde toplanmış.

**Kanal Sayısı:** Orijinal veriler RGB (3 kanal), ancak analiz sırasında Grayscale (1 kanal) formatına dönüştürülmüş.

**Dosya Boyutu:** Dağılım sağa çarpık. Dosyaların büyük kısmı 200-300 KB aralığında, çok azı yüksek boyutlu.

**Sonuç:** Veri seti heterojen olduğu için eğer bir model oluşturacaksak önce tüm görüntüleri sabit bir boyuta (örn. 224x224) yeniden boyutlandırmamız   şart.

## **Görüntü Yükleme ve Görselleştirme**

**Rastgele Görüntüler Seçme Ve Rastgele Görüntülerin İstatistiksel Özellikleri**

In [ ]:

random_samples = train_df.sample(n=9, random_state=42)

# İstatistikleri tutacağımız liste
stats_data = []

# Görselleştirme için figür oluştur (3 satır, 3 sütun - Her hücrede ikili gösterim yapacağız)
plt.figure(figsize=(15, 15))

for idx, (i, row) in enumerate(random_samples.iterrows()):
    path = row['image_path']
    filename = os.path.basename(path)

    # Görüntüyü Oku (OpenCV BGR okur)
    img_bgr = cv2.imread(path)

    # RGB Dönüşümü
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # Grayscale Dönüşümü
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # --- 2.2. İstatistiksel Hesaplamalar ---

    # RGB İstatistikleri (Tüm kanalların ortalaması alınır veya düzleştirilir)
    rgb_mean = np.mean(img_rgb)
    rgb_std = np.std(img_rgb)
    rgb_min = np.min(img_rgb)
    rgb_max = np.max(img_rgb)

    # Gray İstatistikleri
    gray_mean = np.mean(img_gray)
    gray_std = np.std(img_gray)
    gray_min = np.min(img_gray)
    gray_max = np.max(img_gray)

    # Verileri kaydet
    stats_data.append({
        "Image": filename,
        "RGB_Mean": f"{rgb_mean:.2f}", "RGB_Std": f"{rgb_std:.2f}", "RGB_Min": rgb_min, "RGB_Max": rgb_max,
        "Gray_Mean": f"{gray_mean:.2f}", "Gray_Std": f"{gray_std:.2f}", "Gray_Min": gray_min, "Gray_Max": gray_max
    })

    # --- 2.1. Görselleştirme (Yan Yana) ---
    plt.subplot(3, 3, idx + 1)

    # Yan yana göstermek için numpy ile birleştiriyoruz (hstack)
    # Gray görüntüyü 3 kanallı hale getirmeliyiz ki RGB ile yan yana dursun
    img_gray_3ch = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB)
    combined = np.hstack((img_rgb, img_gray_3ch))

    plt.imshow(combined)
    plt.title(f"Örnek {idx+1}\n(Sol: RGB | Sağ: Gray)")
    plt.axis('off')

plt.tight_layout()
plt.show()

# İstatistik Tablosunu Göster
stats_df = pd.DataFrame(stats_data)
print("\n--- 2.2. Rastgele Görüntülerin İstatistiksel Özellikleri ---")
display(stats_df)

**Görsel Karşılaştırma:**

**Yapısal Koruma:** Grayscale dönüşümünde lezyonun sınırları, şekli ve dokusu korunmuştur.

**Veri Kaybı:** Teşhis için kritik öneme sahip olan renk bilgisi (kızarıklık, pigmentasyon tonları) kaybolmuştur.

**İstatistiksel Karşılaştırma:**

**Ortalama (Mean):** RGB ve Grayscale ortalamalarının birbirine çok yakın olması, görüntünün genel parlaklık seviyesinin başarıyla korunduğunu gösterir.

**Standart Sapma ve Min/Max:** Değerlerdeki ufak sapmalar normaldir. 3 farklı renk kanalı (RGB), tek bir kanala (Gri) indirgendiği için uç değerler yumuşamış ve veri sadeleşmiştir.

**Sonuç:** Görüntüler yapısal bütünlüğünü korumuş ancak renk detayı kaybolmuştur. Model eğitimi için bu renk kaybının doğruluğu düşürüp düşürmeyeceği dikkate alınmalıdır.

**Histogram Çizimi (RGB + Grayscale)**

In [ ]:
# Her bir görüntü için histogram çizimi
for idx, (i, row) in enumerate(random_samples.iterrows()):
    path = row['image_path']
    img_bgr = cv2.imread(path)

    if img_bgr is None:
        continue

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # Grafik Alanı Oluştur (1 Satır, 3 Sütun: Görüntü - RGB Hist - Gray Hist)
    fig, axs = plt.subplots(1, 3, figsize=(18, 4))
    fig.suptitle(f"Görüntü {idx+1}: {os.path.basename(path)}", fontsize=14)

    # 1. Görüntünün Kendisi (Referans için)
    axs[0].imshow(img_rgb)
    axs[0].set_title("Orijinal Görüntü")
    axs[0].axis('off')

    # 2. RGB Histogramı (Ayrı Kanallar)
    colors = ('r', 'g', 'b')
    for i, color in enumerate(colors):
        # cv2.calcHist parametreleri: [resim], [kanal], [maske], [bin sayısı], [aralık]
        hist = cv2.calcHist([img_rgb], [i], None, [256], [0, 256])
        axs[1].plot(hist, color=color, label=f'{color.upper()} Kanalı')
        axs[1].set_xlim([0, 256])

    axs[1].set_title("RGB Histogramı")
    axs[1].set_xlabel("Piksel Değeri")
    axs[1].set_ylabel("Piksel Sayısı")
    axs[1].legend()
    axs[1].grid(True, alpha=0.3)

    # 3. Grayscale Histogramı
    hist_gray = cv2.calcHist([img_gray], [0], None, [256], [0, 256])
    axs[2].plot(hist_gray, color='black', label='Gray')
    axs[2].fill_between(range(256), hist_gray.flatten(), color='gray', alpha=0.5) # Altını doldur
    axs[2].set_title("Grayscale Histogramı")
    axs[2].set_xlabel("Piksel Değeri")
    axs[2].set_xlim([0, 256])
    axs[2].grid(True, alpha=0.3)

    plt.show()

**1. RGB Histogramındaki Ayrışma**

Histogramlar incelendiğinde, Kırmızı (R) kanalı diğerlerinden belirgin bir şekilde ayrılarak en sağda (yüksek piksel değerlerinde/aydınlık) konumlanmaktadır. Yeşil (G) ve Mavi (B) kanalları ise daha düşük değerlerde (sol tarafta) ve birbirine daha yakın seyretmektedir.

**Yorum:** Bu durum, görüntülerin insan cildine ait olması ve cilt renginin kırmızı ton ağırlıklı (baskın) olmasından kaynaklanır.

**2. Grayscale Histogramının Yoğunlaşması**

Grayscale histogramı genellikle orta ve yüksek tonlarda (100-200 piksel aralığında) yoğunlaşmaktadır.

**Yorum:** Histogramın sağa (beyaza) yakın olması, görüntülerin genel olarak aydınlık (açık ten rengi) olduğunu gösterir. 0'a yakın (tam siyah) değerlerin azlığı, görüntüde arka planın olmadığını teyit eder.

**3. RGB → Grayscale Dönüşümünün Etkisi**

Dönüşüm, RGB histogramındaki üç ayrı tepe noktasını (peak), insan gözü algısına göre ağırlıklandırarak tek bir dağılım eğrisine indirgemektedir.

**Yorum:** Bu işlem veriyi sadeleştirir (boyut indirgeme). Ancak histogramdaki renk ayrımlarının kaybolması, lezyonun ayırt edici renk özelliklerinin (örneğin melanomdaki mavimsi/kırmızımsı alanlar) parlaklık verisi içinde erimesine neden olmuştur. Genel yapı korunmuş ancak spektral (renk) bilgi kaybedilmiştir.

## **Görüntü İşleme ve İyileştirme**

**Kontrast Germe (Stretching) işlemi**

In [ ]:
def contrast_stretching(img):

    norm_img = cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    return norm_img

# Örnek sayısını alalım
num_samples = len(random_samples)

# Her örnek için 1 satır, 4 sütunluk (Gray Orig, Gray Str, RGB Orig, RGB Str) bir alan açıyoruz
fig, axes = plt.subplots(num_samples, 4, figsize=(20, 5 * num_samples))

# Eğer sadece 1 örnek varsa axes dizisi boyutunu ayarlamamız gerekir
if num_samples == 1:
    axes = axes.reshape(1, -1)

# Veri setindeki her bir görsel için döngü
for i, (index, row) in enumerate(random_samples.iterrows()):
    image_path = row['image_path']

    # 1. Görüntüyü Oku ve Çevir
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        print(f"Hata: {image_path} okunamadı.")
        continue

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # 2. Grayscale Stretching
    gray_stretched = contrast_stretching(img_gray)

    # 3. RGB Stretching (Her kanal ayrı)
    b, g, r = cv2.split(img_bgr)
    b_st = contrast_stretching(b)
    g_st = contrast_stretching(g)
    r_st = contrast_stretching(r)
    rgb_stretched_bgr = cv2.merge([b_st, g_st, r_st])
    rgb_stretched = cv2.cvtColor(rgb_stretched_bgr, cv2.COLOR_BGR2RGB)

    # --- Görselleştirme (i. satır) ---

    # Sütun 0: Orijinal Gray
    axes[i, 0].imshow(img_gray, cmap='gray')
    axes[i, 0].set_title(f"Örnek {i+1} - Orijinal Gray\n(Min:{img_gray.min()}, Max:{img_gray.max()})")

    # Sütun 1: Stretched Gray
    axes[i, 1].imshow(gray_stretched, cmap='gray')
    axes[i, 1].set_title(f"Stretched Gray\n(Min:{gray_stretched.min()}, Max:{gray_stretched.max()})")

    # Sütun 2: Orijinal RGB
    axes[i, 2].imshow(img_rgb)
    axes[i, 2].set_title(f"Örnek {i+1} - Orijinal RGB")

    # Sütun 3: Stretched RGB
    axes[i, 3].imshow(rgb_stretched)
    axes[i, 3].set_title("Stretched RGB (Kanal Bazlı)")

    # Eksenleri kapat
    for j in range(4):
        axes[i, j].axis('off')

plt.tight_layout()
plt.show()

**Kontrast Germe (Contrast Stretching) Karşılaştırması**
**1. Grayscale (Gri) Görüntüler:**

**Netlik Artışı:** Orijinalde dar bir aralığa sıkışmış piksel değerleri 0-255 aralığına yayılarak görüntüdeki "sönük" hava giderilmiştir.

**Detay:** Özellikle Örnek 4 ve 7'de net görüldüğü üzere, kontrast artışı sayesinde lezyonun sınırları ve dokusu sağlıklı deriden çok daha net ayrılmıştır. Gri görüntülerde bu işlem yapısal analizi kolaylaştırmıştır.

**2. RGB (Renkli) Görüntüler:**

**Renk Bozulması:** Her renk kanalı (Kırmızı-Yeşil-Mavi) birbirinden bağımsız gerildiği için kanallar arasındaki oran bozulmuştur.

**Yapaylaşma:** Özellikle Örnek 2 ve 5'te, cildin doğal pembe tonu yapay bir yeşil/sarı renge dönüşmüştür. Tıbbi tanıda lezyonun rengi (kızarıklık vb.) teşhis kriteri olduğu için, bu tür bir renk sapması (distortion) veriyi hatalı hale getirmiştir.

**Genel Sonuç:** Yöntem, gri tonlamalı görüntülerde detay artırımı için oldukça başarılıdır. Ancak renkli görüntülerde doğrudan uygulandığında renk bilgisini bozmaktadır. RGB görüntülerde kontrast artırılacaksa, renkleri korumak için işlem RGB yerine HSV (Sadece Parlaklık/V kanalı) üzerinde yapılmalıdır.

**Histogram Eşitleme (Equalization)**

In [ ]:
# Örnek sayısını alalım
num_samples = len(random_samples)

# Her örnek için 1 satır, 3 sütunluk bir alan (Orijinal, Gray Eq, RGB Y-Channel Eq)
fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))

# Tek örnek varsa boyut hatası almamak için shape düzenlemesi
if num_samples == 1:
    axes = axes.reshape(1, -1)

for i, (index, row) in enumerate(random_samples.iterrows()):
    image_path = row['image_path']

    # Görüntüyü Oku
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        continue

    # BGR -> RGB ve Grayscale dönüşümleri
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # --- 1. Grayscale Histogram Eşitleme ---
    gray_eq = cv2.equalizeHist(img_gray)

    # --- 2. RGB Histogram Eşitleme (YCrCb Yöntemi) ---
    # Amaç: Renk bozulmasını önlemek için sadece parlaklık (Y) kanalını eşitlemek
    img_ycrcb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb)
    y_channel, cr, cb = cv2.split(img_ycrcb)

    # Sadece Y kanalına eşitleme uygula
    y_eq = cv2.equalizeHist(y_channel)

    # Kanalları birleştir ve RGB'ye geri dön
    ycrcb_eq = cv2.merge([y_eq, cr, cb])
    rgb_eq_img = cv2.cvtColor(ycrcb_eq, cv2.COLOR_YCrCb2RGB)

    # --- Görselleştirme (i. satır) ---

    # Sütun 0: Orijinal RGB
    axes[i, 0].imshow(img_rgb)
    axes[i, 0].set_title(f"Örnek {i+1} - Orijinal RGB")

    # Sütun 1: Grayscale Histogram Eşitleme
    axes[i, 1].imshow(gray_eq, cmap='gray')
    axes[i, 1].set_title("Gray Histogram Eq")

    # Sütun 2: RGB Histogram Eşitleme (Y Kanalı)
    axes[i, 2].imshow(rgb_eq_img)
    axes[i, 2].set_title("RGB Histogram Eq (Y-Kanalı)")

    # Eksenleri kapat
    for j in range(3):
        axes[i, j].axis('off')

plt.tight_layout()
plt.show()

**1. Gray Histogram Eq (Ortadaki Sütun)**

**Maksimum Kontrast:** Görüntüdeki gri ton dağılımı tüm histograma zorla yayıldığı için kontrast dramatik şekilde artmıştır.

**Detay ve Gürültü Patlaması:** Lezyon sınırları çok keskinleşmiştir. Ancak bunun yan etkisi olarak ciltteki kıllar, gözenekler ve arka plan gürültüleri de aşırı belirginleşmiştir (Özellikle Örnek 2 ve 8). Görüntü "sert" ve yapay bir dokuya bürünmüştür.

**2. RGB Histogram Eq (Y-Kanalı) (Sağdaki Sütun)**

**Renk Korunumu:** Bir önceki analizdeki (stretching) "yeşil/sarı cilt" sorunu burada yoktur. Çünkü renk (kroma) bilgisine dokunulmadan sadece ışık şiddetiyle oynanmıştır. Cilt tonları hala kırmızı/pembe eksenindedir.

**Aşırı Doygunluk (Oversaturation):** Parlaklık kontrastı aşırı artırıldığı için renkler çok daha koyu ve yoğun görünmektedir. Lezyonlar olduğundan daha koyu ve "yanık" gibi algılanmaktadır.

**Genel Sonuç ve Karşılaştırma**

Histogram Eşitleme, "Kontrast Germe"ye göre çok daha sert bir işlemdir.

**Avantajı:** Lezyon sınırlarını (segmentasyon için) belirginleştirmekte çok güçlüdür. Görülmesi zor olan silik lezyonları (Örnek 5) ortaya çıkarır.

**Dezavantajı:** Görüntüye çok fazla "yapay gürültü" ekler ve doğal görünümü bozar. Tıbbi teşhiste modelin doku algısını şaşırtabilir, ancak sınır tespiti (sınır çizme) görevlerinde işe yarayabilir.

**Gamma Düzeltme**

In [ ]:
def adjust_gamma(image, gamma=1.0):
    """
    Lookup Table (LUT) kullanarak hızlı Gamma Düzeltmesi
    """
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

gamma_values = [0.5, 1.0, 2.0]
num_samples = len(random_samples)

# Her örnek için 1 satır, 6 sütun (3 RGB, 3 Gray)
# Genişlik (width) 24 yaparak görsellerin sıkışmasını önlüyoruz
fig, axes = plt.subplots(num_samples, 6, figsize=(24, 4 * num_samples))

if num_samples == 1:
    axes = axes.reshape(1, -1)

for i, (index, row) in enumerate(random_samples.iterrows()):
    image_path = row['image_path']

    # Okuma ve Dönüştürme
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        continue

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # Gamma değerleri üzerinde döngü
    for j, gamma in enumerate(gamma_values):
        # --- RGB Uygulama ---
        res_rgb = adjust_gamma(img_rgb, gamma=gamma)

        # İlk 3 sütun (0, 1, 2) RGB sonuçları için
        axes[i, j].imshow(res_rgb)
        axes[i, j].set_title(f"Örnek {i+1} - RGB \nGamma={gamma}")
        axes[i, j].axis('off')

        # --- Grayscale Uygulama ---
        res_gray = adjust_gamma(img_gray, gamma=gamma)

        # Sonraki 3 sütun (3, 4, 5) Gray sonuçları için
        axes[i, j + 3].imshow(res_gray, cmap='gray')
        axes[i, j + 3].set_title(f"Gray \nGamma={gamma}")
        axes[i, j + 3].axis('off')

plt.tight_layout()
plt.show()


**1. Parlaklık Değişimleri (Gözlem)**

**Gamma = 0.5 (Düşük Gamma):** Görüntülerin piksel değerleri koyulaşmış, renkler daha yoğun ve doygun hale gelmiştir.

**Gamma = 2.0 (Yüksek Gamma):** Görüntüler aşırı parlamış ve beyazlaşmıştır. Sanki görüntünün üzerine "sisli" bir katman gelmiş gibi soluklaşmıştır.

**2. Görüntü Kalitesi Üzerindeki Etkisi**

**Detay Kaybı (Koyu Alanlar):** Gamma 0.5'te görüntü koyulaştığı için, lezyonun içindeki koyu renkli dokusal detaylar siyaha boğulmuş ve kaybolmuştur. (Örn: Örnek 6).

**Kontrast Kaybı (Açık Alanlar):** Gamma 2.0'da görüntü aşırı aydınlandığı için ("washed out" etkisi), lezyon ile sağlıklı deri arasındaki kontrast azalmıştır. Lezyon sınırları silikleşmiştir.

**Sonuç:** Her iki uç değer de veri kaybına neden olmuştur. Gamma = 1.0 (Orijinal), hem parlaklık dengesi hem de doku detaylarının korunması açısından en ideal kaliteyi sunmaktadır.

## **Gürültü Azaltma**

In [ ]:
# Filtre Parametresi (Kernel Size)
k_size = 5

num_samples = len(random_samples)

# Her örnek için 1 satır, 6 sütun
# Sütunlar: [RGB Orig, RGB Median, RGB Gaussian] | [Gray Orig, Gray Median, Gray Gaussian]
fig, axes = plt.subplots(num_samples, 6, figsize=(24, 4 * num_samples))

if num_samples == 1:
    axes = axes.reshape(1, -1)

for i, (index, row) in enumerate(random_samples.iterrows()):
    image_path = row['image_path']

    # Okuma ve Dönüştürme
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        continue

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # --- 4.1. Median Blur ---
    median_rgb = cv2.medianBlur(img_rgb, k_size)
    median_gray = cv2.medianBlur(img_gray, k_size)

    # --- 4.2. Gaussian Blur ---
    gaussian_rgb = cv2.GaussianBlur(img_rgb, (k_size, k_size), 0)
    gaussian_gray = cv2.GaussianBlur(img_gray, (k_size, k_size), 0)

    # --- Görselleştirme ---

    # 1. RGB - Orijinal
    axes[i, 0].imshow(img_rgb)
    axes[i, 0].set_title(f"Örnek {i+1} - RGB Orijinal")

    # 2. RGB - Median
    axes[i, 1].imshow(median_rgb)
    axes[i, 1].set_title(f"RGB Median (k={k_size})")

    # 3. RGB - Gaussian
    axes[i, 2].imshow(gaussian_rgb)
    axes[i, 2].set_title(f"RGB Gaussian (k={k_size})")

    # 4. Gray - Orijinal
    axes[i, 3].imshow(img_gray, cmap='gray')
    axes[i, 3].set_title("Gray Orijinal")

    # 5. Gray - Median
    axes[i, 4].imshow(median_gray, cmap='gray')
    axes[i, 4].set_title(f"Gray Median (k={k_size})")

    # 6. Gray - Gaussian
    axes[i, 5].imshow(gaussian_gray, cmap='gray')
    axes[i, 5].set_title(f"Gray Gaussian (k={k_size})")

    for j in range(6):
        axes[i, j].axis('off')

plt.tight_layout()
plt.show()

Görsellerde temel problem (gürültü), lezyon üzerindeki kıllar ve ince doku detaylarıdır.

**1. RGB (Renkli) Görüntüler İçin Karşılaştırma**

**Median Blur (k=5):**

**Kenar Koruma:** Evet, görselde açıkça görülüyor ki Median filtre kenarları çok daha iyi koruyor. Özellikle lezyonun deri ile birleştiği sınırlara bakarsanız (Örneğin 6. ve 7. satırdaki örnekler), geçişler hala keskin.

**Detay/Gürültü:** Median filtre, "tuz-biber" gürültüsü gibi davranan kılları (örneğin 3. satır) yok etmede çok başarılı. Ancak bunu yaparken dokuyu biraz "plastikleştiriyor" veya yağlı boya tablosu gibi düzleştiriyor.

**Gaussian Blur (k=5):**

**Detay Kaybı:** Evet, detay kaybına neden oluyor. Görüntünün tamamına homojen bir bulanıklık (blur) uyguladığı için, hem kıllar hem de lezyonun kendi içindeki pigment dokusu birbirine karışıyor.

**Gözlem:** Kılları tamamen yok etmek yerine onları "flu" hale getiriyor (Örneğin 3. satır RGB Gaussian). Bu da lezyon sınırlarının yumuşamasına ve analiz yaparken alanın net seçilememesine neden olabilir.

**2. Gray (Gri Tonlamalı) Görüntüler İçin Karşılaştırma**

Gri tonlamalı görsellerde kontrast farkı daha belirgin olduğu için filtrelerin etkisi daha net görülmektedir.

**Median Blur (k=5):**

Kenar Koruma: Gri tonlamada bu etki daha bariz. Lezyonun koyu tonu ile derinin açık tonu arasındaki sınır (kontrast geçişi) korunmuş.

**Kıl Temizleme:** Siyah kıllar, gri arka planda "impulse noise" (ani pik yapan gürültü) gibi davrandığı için Median filtre bunları matematiksel olarak çok iyi ekarte etmiş. Kılların olduğu yerler çevre dokunun rengini alarak kapanmış.

**Gaussian Blur (k=5):**

**Detay Kaybı:** Gri görüntülerde Gaussian çıktısı "sisli" görünüyor. İnce detaylar (deri çizgileri ve lezyon içi noktalar) kaybolmuş.

**Sonuç:** Siyah kıllar tamamen kaybolmamış, gri gölgeler (smearing) olarak kalmış. Bu, görüntü işleme algoritmalarında "leke" olarak algılanmaya devam edebilir.


**Median blur kenarları daha iyi korur mu?**

Evet, kesinlikle. Hem RGB hem de Gray görsellerde, Median filtre lezyonun dış sınırlarını keskin tutarken içindeki kılları temizlemiştir. Gaussian filtre ise sınırları yumuşatmış (bulanıklaştırmış)tır.

**Gaussian blur detay kaybına neden oluyor mu?**

 Evet. Gaussian filtre, görüntüdeki yüksek frekanslı bileşenleri (keskin detayları) bastırarak görüntüyü genel olarak flulaştırır. Görsellerde lezyon içindeki pigment ağının ve dokunun Gaussian ile daha çok kaybolduğu, Median ile ise daha düz ama keskin kaldığı görülmektedir.

Median Blur bu veri seti (kıllı deri görüntüleri) için Gaussian'dan çok daha üstün bir sonuç vermektedir.

## **Döndürme ve Ayna Çevirme (Flipping)**

In [ ]:
def rotate_image(image, angle):
    """
    Bir görüntüyü merkezi etrafında verilen açıda döndürür.
    """
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    # Döndürme matrisini hesapla
    M = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Dönüşümü uygula (Kenarlarda oluşan boşluklar siyah/0 ile doldurulur)
    rotated = cv2.warpAffine(image, M, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0))
    return rotated

num_samples = len(random_samples)

# Her örnek için 1 satır, 6 sütun (3 RGB, 3 Gray)
# Sütunlar: RGB(Orijinal, Rotate, Flip) | Gray(Orijinal, Rotate, Flip)
fig, axes = plt.subplots(num_samples, 6, figsize=(24, 4 * num_samples))

if num_samples == 1:
    axes = axes.reshape(1, -1)

for i, (index, row) in enumerate(random_samples.iterrows()):
    image_path = row['image_path']
    filename = os.path.basename(image_path)

    # 1. Okuma
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        continue

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # 2. İşlemler
    # Her resim için 0-15 derece arasında benzersiz rastgele bir açı
    random_angle = np.random.uniform(0, 15)

    # Döndürme
    rotated_rgb = rotate_image(img_rgb, random_angle)
    rotated_gray = rotate_image(img_gray, random_angle)

    # Yatay Çevirme (Flip)
    flipped_rgb = cv2.flip(img_rgb, 1)
    flipped_gray = cv2.flip(img_gray, 1)

    # --- Görselleştirme ---

    # RGB Sütunları
    axes[i, 0].imshow(img_rgb)
    axes[i, 0].set_title(f"Örnek {i+1} - RGB Orijinal")

    axes[i, 1].imshow(rotated_rgb)
    axes[i, 1].set_title(f"RGB Döndürme ({random_angle:.1f}°)")

    axes[i, 2].imshow(flipped_rgb)
    axes[i, 2].set_title("RGB Yatay Flip")

    # Gray Sütunları
    axes[i, 3].imshow(img_gray, cmap='gray')
    axes[i, 3].set_title("Gray Orijinal")

    axes[i, 4].imshow(rotated_gray, cmap='gray')
    axes[i, 4].set_title(f"Gray Döndürme ({random_angle:.1f}°)")

    axes[i, 5].imshow(flipped_gray, cmap='gray')
    axes[i, 5].set_title("Gray Yatay Flip")

    # Eksenleri kapat
    for j in range(6):
        axes[i, j].axis('off')

plt.tight_layout()
plt.show()

**1. Yatay Çevirme (Horizontal Flip) ve Simetri Gözlemi**

Bu işlem, görüntünün dikey bir eksen etrafında aynalanmasıdır. Dermatolojideki ABCD kuralının "A"sı (Asymmetry) için en kritik gözlem buradadır.

**Asimetrik Örnekler (Simetrisi Bozuk Olanlar):**

**Örnek 6:** En bariz simetri farkı burada. Orijinal görüntüde lezyon sol-alt köşeden sağ-üst köşeye doğru uzanıyor. "Yatay Flip" yapıldığında ise tam tersine sağ-alt köşeden sol-üst köşeye uzanıyor. Bu lezyonun şekilsel olarak asimetrik olduğunu kanıtlar.

**Örnek 4:** Orijinalde lezyonun yoğun kısmı (koyu bölge) hafif sağa meyilli. Flip işleminde bu yoğunluk sola geçiyor. Şekil tam bir daire olmadığı için aynalama işlemi görüntüyü belirgin şekilde değiştiriyor.

**Örnek 2:** Dağınık lekelenmeler (satellite lesions) orijinalde belli noktalardayken, flip işleminde yer değiştiriyor. Bu tür dağınık yapılar asimetriyi artırır.

**Simetrik (veya Simetriye Yakın) Örnekler:**

**Örnek 3:** Lezyon oldukça yuvarlak ve merkeze yakın. Flip yapıldığında (aynalandığında) görüntü orijinaline çok benziyor. Bu, lezyonun yüksek oranda simetrik olduğunu gösterir.

**Örnek 1:** Kenarları biraz düzensiz olsa da genel hatlarıyla yuvarlak olduğu için flip işlemi görsel algıyı radikal biçimde değiştirmiyor.

**Yorum:** Bir lezyon ne kadar asimetrikse, "Horizontal Flip" işlemi o kadar "yeni" ve "farklı" bir görüntü üretir. Bu durum, yapay zeka modelinin şekil varyasyonlarını öğrenmesi (genelleme yeteneği) için çok değerlidir.

**2. Döndürme (Rotation) Etkisi**

Köşe Kayıpları (Siyah Alanlar): Döndürme işleminde (örneğin Örnek 2'de 11.3 derece), kare resim döndüğü için köşelerde veri kaybı oluşmuş ve siyah (veya siyah/gri dolgu) üçgenler ortaya çıkmış.

**Simetriye Etkisi:** Döndürme işlemi, lezyonun kendi içindeki simetrisini değiştirmez, sadece uzaydaki yönünü değiştirir.

**Örnek 9 (Doku Analizi):** Bu örnekte çizgisel bir deri dokusu var.

**Orijinal:** Çizgiler yatay/hafif diyagonal.

**Döndürme (13.6°):** Çizgilerin açısı dikleşmiş.

**Flip:** Çizgilerin yönü tam tersine dönmüş. Bu, doku tabanlı analiz yapan algoritmalar için önemli bir çeşitliliktir.

**3. RGB vs Gray (Gri) Karşılaştırması**

**Simetri Algısı:** Gri tonlamalı görsellerde renk bilgisi (kırmızılık, kahverengilik) kaybolduğu için sadece yapısal simetriye (şekil ve doku) odaklanmak daha kolaydır.

Özellikle Örnek 6'nın gri versiyonlarına bakıldığında, lezyonun geometrik asimetrisi renkli versiyona göre daha net (dikkat dağıtıcı renkler olmadan) seçilebilmektedir.

**Sonuç:** Görsellerdeki "Yatay Flip" sütunu, lezyonların simetrisini test etmek için bir ayna görevi görmektedir. Örnek 6 ve Örnek 4 gibi asimetrik lezyonlar, aynalandıklarında orijinal hallerinden belirgin şekilde farklılaşırken, Örnek 3 gibi simetrik lezyonlar neredeyse aynı kalmaktadır.

## **Frekans Alanında Filtreleme (FFT)**

In [ ]:
# Örnek sayısını alalım
num_samples = len(random_samples)

# Her örnek için 1 satır, 4 sütun
# Sütunlar: [Orijinal Gray] | [Spektrum] | [Maske] | [Sonuç (LPF)]
fig, axes = plt.subplots(num_samples, 4, figsize=(20, 5 * num_samples))

# Tek örnek varsa boyut hatası almamak için shape düzenlemesi

for i, (index, row) in enumerate(random_samples.iterrows()):
    image_path = row['image_path']

    # 1. Okuma ve Grayscale Dönüşümü
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        continue
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # --- 6.1. Fourier Dönüşümü (FFT) ---
    f = np.fft.fft2(img_gray)
    fshift = np.fft.fftshift(f)

    # Görselleştirme için Magnitude Spectrum (Log scale)
    # +1 ekliyoruz ki log(0) hatası almayalım
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1)

    # --- 6.2. Low Pass Filter (LPF) Maskesi ---
    rows, cols = img_gray.shape
    crow, ccol = rows // 2, cols // 2

    mask = np.zeros((rows, cols), np.uint8)
    r = 40  # Yarıçap (Değiştirilebilir: 30 daha bulanık, 60 daha net yapar)
    cv2.ellipse(mask, (ccol, crow), (r, r), 0, 0, 360, 1, -1)

    # Maskeyi uygula
    fshift_filtered = fshift * mask

    # --- 6.3. Ters Fourier Dönüşümü (Inverse FFT) ---
    f_ishift = np.fft.ifftshift(fshift_filtered)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back)

    # --- Görselleştirme ---

    # 1. Orijinal
    axes[i, 0].imshow(img_gray, cmap='gray')
    axes[i, 0].set_title(f"Örnek {i+1} - Orijinal Gray")

    # 2. Spektrum
    axes[i, 1].imshow(magnitude_spectrum, cmap='gray')
    axes[i, 1].set_title("Frekans Spektrumu")

    # 3. Maske
    axes[i, 2].imshow(mask, cmap='gray')
    axes[i, 2].set_title(f"LPF Maskesi (r={r})")

    # 4. Sonuç
    axes[i, 3].imshow(img_back, cmap='gray')
    axes[i, 3].set_title("Sonuç (Filtrelenmiş)")

    # Eksenleri kapat
    for j in range(4):
        axes[i, j].axis('off')

plt.tight_layout()
plt.show()

**1. Temel Fark: Keskinlik ve Bulanıklık (Sharpness vs. Blur)**

**•	Orijinal Görüntüler:** Orijinal resimlerdeki detaylar keskindir. Deri üzerindeki gözenekler, ince çizgiler ve leke sınırları net bir şekilde seçilebilir. Bu detaylar, frekans düzleminde yüksek frekanslı bileşenlere karşılık gelir.

**•	Sonuç (Filtrelenmiş):** Alçak Geçiren Filtre (LPF), görüntünün merkezindeki düşük frekansları geçirip, kenarlardaki yüksek frekansları kestiği (maskelediği) için sonuç görüntüsü bulanıklaşmıştır (smoothing). Görüntü sanki buzlu bir camın arkasından bakıyormuşsunuz gibi yumuşak bir hal almıştır.

**2. İstenmeyen Nesnelerin (Gürültü ve Kıllar) Kaybolması**

Bu işlemin medikal görüntülemede (örneğin dermoskopi) en yaygın kullanım amaçlarından biri gürültü azaltmadır.

**•	Kıllar ve İnce Detaylar:** Özellikle 3. ve 7. satırdaki örneklere dikkat ederseniz, orijinal resimde lekenin üzerinde belirgin siyah kıllar/tüyler bulunmaktadır. Bu kıllar, görüntüde ani renk değişimleri yarattığı için "yüksek frekans" olarak kabul edilir.

**•	Filtre Etkisi:** LPF maskesi (r=40 yarıçapı ile) bu yüksek frekansları kestiği için, sonuç görüntülerinde kıllar neredeyse tamamen kaybolmuş veya çok silikleşmiştir. Bu, lezyonun ana şeklini (segmentasyon için) ortaya çıkarmak adına yararlıdır ancak doku detayını kaybettirir.

**3. Maske Yapısının Etkisi (Ringing Effect / Çınlama)**

•	Kullanılan maske (3. sütun), keskin sınırları olan beyaz bir dairedir (Ideal Low Pass Filter).

•	Frekans uzayında bu kadar keskin bir kesme işlemi (hard cut-off), uzaysal düzleme geri dönüldüğünde (Inverse FFT sonrası), bazen nesnelerin etrafında hafif dalgalanmalar veya halkalanmalar (ringing artifacts) yaratabilir. Sonuç görüntülerine çok yakından bakıldığında leke sınırlarının orijinali kadar doğal değil, hafif dalgalı bir geçişe sahip olduğu görülebilir.

**4. Genel Kontrast ve Aydınlık**

•	Filtreleme işlemi "DC bileşeni" (frekans spektrumunun tam merkezi, yani ortalama parlaklık) koruduğu için, görüntülerin genel parlaklık seviyesi ve ana lekenin koyuluğu (kontrast dengesi) büyük ölçüde korunmuştur. Sadece geçişler yumuşamıştır.


**Özet**

**Yapılan işlem şudur:**

Orijinal Görüntü $\rightarrow$ FFT (Frekanslara ayır) $\rightarrow$ Maske (Yüksek frekanslı detayları/kılları/gürültüyü at) $\rightarrow$ Inverse FFT (Geri dön).

**Sonuç:** Görüntüdeki "gürültü" (noise) ve "keskin detaylar" (high frequency) temizlenmiş, ancak bunun karşılığında görüntü netliğini kaybederek flulaşmıştır.



## **Keskinleştirme ve Enterpolasyon**

In [ ]:
# --- FONKSİYON: Unsharp Masking ---
def apply_unsharp_mask(image, kernel_size=(9, 9), sigma=10.0, weight=3):
    """
    Görüntüye Gaussian Blur uygulayıp, orijinal görüntü ile ağırlıklı olarak
    toplayarak keskinleştirme yapar.
    """
    blurred = cv2.GaussianBlur(image, kernel_size, sigma)
    # Formül: sharpened = original * weight + blurred * (1 - weight)
    sharpened = cv2.addWeighted(image, weight, blurred, 1.0 - weight, 0)
    return sharpened

num_samples = len(random_samples)
scale_factor = 2  # 2 kat büyütme

# Her örnek için 1 satır, 6 sütun
# Sütunlar: [RGB Orig, RGB Sharp, RGB Zoom] | [Gray Orig, Gray Sharp, Gray Zoom]
fig, axes = plt.subplots(num_samples, 6, figsize=(24, 4 * num_samples))

if num_samples == 1:
    axes = axes.reshape(1, -1)

for i, (index, row) in enumerate(random_samples.iterrows()):
    image_path = row['image_path']

    # 1. Okuma
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        continue

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # --- 7.1. Keskinleştirme (Unsharp Mask) ---
    sharp_rgb = apply_unsharp_mask(img_rgb)
    sharp_gray = apply_unsharp_mask(img_gray)

    # --- 7.2. Bicubic Upscale (Büyütme) ---
    upscaled_rgb = cv2.resize(sharp_rgb, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
    upscaled_gray = cv2.resize(sharp_gray, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)

    # --- Kırpma (Cropping) ---
    # Büyütülmüş resmin sadece ortasını alalım ki detay farkı görülsün
    h, w, _ = upscaled_rgb.shape
    crop_rgb = upscaled_rgb[h//4 : 3*h//4, w//4 : 3*w//4]

    h_g, w_g = upscaled_gray.shape
    crop_gray = upscaled_gray[h_g//4 : 3*h_g//4, w_g//4 : 3*w_g//4]

    # --- Görselleştirme ---

    # RGB Sütunları
    axes[i, 0].imshow(img_rgb)
    axes[i, 0].set_title(f"Örnek {i+1} - RGB Orijinal")

    axes[i, 1].imshow(sharp_rgb)
    axes[i, 1].set_title("RGB Unsharp Mask")

    axes[i, 2].imshow(crop_rgb)
    axes[i, 2].set_title("RGB Upscaled (Crop)")

    # Gray Sütunları
    axes[i, 3].imshow(img_gray, cmap='gray')
    axes[i, 3].set_title("Gray Orijinal")

    axes[i, 4].imshow(sharp_gray, cmap='gray')
    axes[i, 4].set_title("Gray Unsharp Mask")

    axes[i, 5].imshow(crop_gray, cmap='gray')
    axes[i, 5].set_title("Gray Upscaled (Crop)")

    for j in range(6):
        axes[i, j].axis('off')

plt.tight_layout()
plt.show()

**1. Keskinleştirilmiş (Unsharp Mask) RGB ve Grayscale Görüntülerin Karşılaştırması**

Bu sütunlarda (RGB Unsharp Mask ve Gray Unsharp Mask), orijinal görüntülere "Unsharp Masking" (Keskinleştirme Maskesi) filtresi uygulanarak kenar belirginliğinin artırıldığı görülmektedir.

**RGB Unsharp Mask (Renkli Keskinleştirme):**

**Avantajı:** Renk bilgisi korunduğu için lezyonun tanısal açıdan kritik olan renk varyasyonları (kırmızı, kahverengi, mavi tonlar) belirginleşmiştir. Özellikle Örnek 2 ve Örnek 3'teki damarsal yapılar ve pigment ağları, orijinal haline göre çok daha "net" ve ayrışmış durmaktadır.

**Gözlem:** Keskinleştirme işlemi, lezyonun dokusunu ön plana çıkarmış ancak aynı zamanda cilt üzerindeki kıllar ve ince çizgiler gibi "gürültü" sayılabilecek unsurları da daha görünür kılmıştır.

**Gray Unsharp Mask (Gri Tonlamalı Keskinleştirme):**

**Avantajı:** Renk bilgisinin dikkat dağıtıcı etkisi ortadan kalktığı için sadece "Luminance" (Parlaklık) üzerindeki kontrast değişimlerine odaklanılır. Bu, lezyonun sınırlarını (border) ve yapısal desenlerini (ağ yapıları, globüller) analiz etmek için daha yüksek bir kontrast sunar.

**Fark:** RGB versiyona göre, gri tonlamalı keskinleştirme işleminde dokular daha sert ve "granüllü" görünmektedir. Örneğin Örnek 5 ve Örnek 8'de gri versiyon, lezyonun merkezindeki yoğunluğu RGB'ye göre daha dramatik bir şekilde vurgulamaktadır.

**Sonuç:** RGB keskinleştirme insan gözüyle tanı (doktor incelemesi) için daha doğal ve bilgilendiriciyken; Gray keskinleştirme, bilgisayarlı görü (makine öğrenmesi) algoritmalarının kenar tespiti (edge detection) ve segmentasyon yapması için daha uygun bir zemin hazırlamıştır.

**2. Enterpolasyon (Upscaled Crop) Sonrası Kalite Yorumu**

"Upscaled (Crop)" sütunlarında, görüntülerin belirli bir bölgesinin kesilip çözünürlüğünün artırıldığı (enterpolasyon yapıldığı) görülmektedir.

**Pikselleşme (Pixelation) Durumu:**

Görüntüler yaklaştırıldığında belirgin bir "kareleşme" veya bloklaşma görülmemektedir. Bu, enterpolasyon sırasında basit bir "Nearest Neighbor" (En yakın komşu) yerine, Bilinear veya daha büyük olasılıkla Bicubic gibi yumuşatma yapan bir algoritma kullanıldığını işaret eder. Kenarlar keskin kırılmalar yerine yumuşak geçişlere sahiptir.

**Detay Kaybı ve Bulanıklık:**

Yaklaştırma işlemi kaçınılmaz olarak yüksek frekanslı detayların (çok ince dokuların) kaybına yol açmıştır. Unsharp Mask sütunundaki o "gevrek" ve keskin görüntü, Upscaled sütununda yerini daha "soft" (yumuşak) ve hafif bulanık bir dokuya bırakmıştır.

Özellikle Örnek 3 ve Örnek 7'deki detaylı bölgelere bakıldığında (kabuklanma veya yoğun pigmentasyon alanları), upscaling sonrası doku bilgisinin biraz "düzleştiği" görülmektedir.

**Tanısal Kullanılabilirlik:**

Kalite kaybına rağmen, dermatoskopik yapıların (globüller, çizgilenmeler) ana karakteristiği korunmuştur. Örnek 6'daki lezyonun sınır yapısı veya Örnek 9'daki sıvı/kabuk dokusu, büyütülmüş versiyonda hala ayırt edilebilir durumdadır.

**Özetle:** Enterpolasyon işlemi, görüntüyü büyütürken pikselleşmeyi başarıyla engellemiş ancak keskinlikten ödün vererek görüntüyü yumuşatmıştır. Dijital patolojide bu tür bir upscaling, genel yapıyı incelemek için yeterli olsa da, hücresel düzeyde inceleme için orijinal çözünürlüğün yüksek olması tercih edilir.

#**FFT Sonuçlarının Özel Olarak Yorumlanması**

### 1. Merkezdeki Parlak Nokta (DC Bileşeni)
Her bir spektrumun tam ortasında çok parlak beyaz bir nokta veya öbek görürsünüz.

* **Nedir:** Burası (0,0) frekansıdır, yani DC bileşeni.
* **Anlamı:** Görüntünün ortalama parlaklığını temsil eder. Görüntülerinizin çoğu açık renkli bir deriden oluştuğu için, toplam ışık enerjisinin çok büyük bir kısmı burada toplanır. Bu yüzden merkez her zaman en parlak kısımdır.

### 2. Işığın Dağılımı (Yüksek vs. Alçak Frekans)
Merkezden dışarıya doğru gidildikçe frekans artar.

* **Merkeze Yakın Alanlar:** Görüntüdeki yavaş renk geçişlerini (derinin genel tonu, lekenin ana gövdesi) temsil eder. Dikkat ederseniz tüm örneklerde enerji merkezde yoğundur; çünkü görüntüleriniz ağırlıklı olarak yumuşak dokulardan oluşur.
* **Dış Kenarlar:** Görüntüdeki keskin değişimleri (kıllar, sert kenarlar, gürültü) temsil eder.

### 3. "Yıldız" veya "Çizgi" Şeklindeki Parlamalar (Yönelim İlişkisi)
FFT spektrumunun en karakteristik özelliği, uzaysal düzlemdeki (orijinal resimdeki) çizgilerin yönüne dik (90 derece) tepki vermesidir.

**Örnek 3 ve 7'ye Dikkat Edin (Kıllar):**
* **Orijinal:** Görüntüde çok sayıda çapraz ve karmaşık kıllar var.
* **Spektrum:** Merkezden dışarıya doğru saçılan, havai fişek gibi çizgiler görürsünüz. Bu çizgiler, kılların oluşturduğu "yüksek frekanslı" ani geçişlerdir.

**Örnek 9'a Dikkat Edin (Cetvel İzleri):**
* **Orijinal:** Sol üst köşede cetvel çizgileri (siyah, düzenli çizgiler) var.
* **Spektrum:** Spektrumun sol alt ve sağ üst köşesine doğru uzanan çok belirgin, kesik kesik çizgiler (noktalar dizisi) oluşmuştur. Bu, periyodik gürültünün (düzenli tekrar eden çizgilerin) frekans uzayındaki imzasıdır.

### 4. Simetri
Tüm frekans spektrumu görüntüleri merkeze göre simetriktir (Point Symmetry).

* Bunun nedeni, giriş görüntüsünün (orijinal resmin) reel sayılardan oluşmasıdır. Matematiksel olarak reel bir sinyalin Fourier dönüşümü "Hermitian simetrisi" gösterir. Yani spektrumun sağ üstünde ne görüyorsanız, sol altında da aynısını görürsünüz.

### 5. Neden LPF (Alçak Geçiren Filtre) İşe Yarıyor?
3. sütundaki "LPF Maskesi"nin (beyaz daire) mantığı tam olarak bu spektrum görüntülerine dayanır:

* Maske, sadece merkezdeki parlak yoğunluğu (leke ve deri bilgisini) alır.
* Maskenin siyah kısımları, merkezin uzağına saçılmış olan o "çizgileri", "yıldız saçılmalarını" (yani kılları ve cetvel izlerini) keser.
* Böylece Ters FFT yapıldığında (4. sütun), kılların frekans bilgisi silindiği için görüntüde sadece bulanık leke kalır.

# **Tüm Sonuçlara Göre RGB ve Grayscale İşlemlerinin Karşılaştırmalı Analizi**

Bu çalışmada, ISIC veri seti üzerindeki deri lezyonu görüntüleri hem renkli (RGB) hem de gri tonlamalı (Grayscale) formatlarda işlenmiş; görüntü iyileştirme, gürültü azaltma ve yapısal analiz yöntemlerinin her iki format üzerindeki etkileri karşılaştırılmıştır. Elde edilen bulgular aşağıda özetlenmiştir.

### 1. Veri Yapısı ve Bilgi Korunumu
* **Renksel Bilgi:** RGB formatı, melanom teşhisi için kritik öneme sahip olan pigmentasyon tonlarını ve kızarıklık bilgisini korumaktadır .
* **Yapısal Koruma:** Grayscale dönüşümü, 3 kanallı veriyi tek kanala indirgeyerek veriyi sadeleştirmiş (boyut indirgeme); lezyonun sınırlarını, şeklini ve dokusunu başarıyla korurken renk ayrımını kaybetmiştir .

### 2. Görüntü İyileştirme Tekniklerinin Etkisi
Görüntü kalitesini artırmak için uygulanan tekniklerde iki format farklı tepkiler vermiştir:
* **Kontrast Germe (Stretching):** Grayscale görüntülerde piksel değerlerini yayıp detayları ve netliği belirgin şekilde artırırken ; RGB görüntülerde kanallar birbirinden bağımsız gerildiği için renk dengesi bozulmuş, cilt tonunun yapay (yeşil/sarı) bir renge dönüşmesine neden olmuştur .
* **Histogram Eşitleme:** Grayscale üzerinde lezyon sınırlarını (segmentasyon için) güçlendirmiş ancak gürültüyü artırmıştır . RGB üzerinde ise Y-Kanalı (Parlaklık) eşitlendiğinde renkler korunmuş ancak aşırı doygunluk (oversaturation) oluşarak lezyonlar olduğundan daha koyu ve "yanık" görünmüştür .
* **Gamma Düzeltme:** Her iki formatta da Gamma=1.0 (Orijinal) değeri en dengeli sonucu vermiş; düşük gamma detay kaybına, yüksek gamma ise kontrast kaybına yol açmıştır .

### 3. Gürültü Azaltma ve Doku Analizi
Deri üzerindeki kıllar ve ince doku detayları "gürültü" olarak ele alındığında:
* **Filtre Performansı:** Median Blur (k=5), hem RGB hem de Grayscale görüntülerde lezyon sınırlarını koruyarak kılları temizlemede Gaussian Blur'a göre çok daha üstün performans göstermiştir .
* **Frekans Alanı (FFT):** Alçak Geçiren Filtre (LPF), yüksek frekanslı bileşenler olan kılları başarıyla maskelemiş ancak görüntüyü genel olarak flulaştırmıştır .

### 4. Geometrik ve Yapısal Değerlendirme (ABCD Kuralı)
* **Simetri Tespiti:** Yatay çevirme (Horizontal Flip) işlemi, lezyonların asimetrisini (ABCD kuralının 'A'sı) tespit etmek için etkili bir yöntemdir . Grayscale görüntüler, dikkat dağıtıcı renk bilgisi olmadığı için geometrik asimetriyi analiz etmede daha net bir zemin sunmaktadır .
* **Kenar Tespiti:** Keskinleştirme (Unsharp Mask) işleminde Grayscale formatı, algoritmik kenar bulma (edge detection) ve segmentasyon görevleri için daha yüksek kontrast sağlamıştır .

### Genel Değerlendirme ve Sonuç
Yapılan analizler ışığında;
1. **Sınıflandırma Modelleri İçin:** Melanom belirtileri renk değişimlerine bağlı olduğundan, derin öğrenme (CNN) modellerinin eğitiminde **RGB** verisinin kullanılması elzemdir .
2. **Segmentasyon ve Ön İşleme İçin:** Sınır çizme, şekil analizi ve gürültü temizleme (kıl kaldırma) adımlarında işlem maliyetini düşürmek ve yapıya odaklanmak için **Grayscale** verisi daha başarılıdır .